In [81]:
import os

project_root = "/Users/nimisha/Desktop/learning/news-reliability-detector/"
os.chdir(project_root)

In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, udf, concat
from pyspark.sql.types import StringType
from pyspark.ml.feature import (
    HashingTF,
    IDF,
    Tokenizer,
    VectorAssembler,
    StopWordsRemover,
)
from pyspark.ml import Pipeline

# local modules
from src.load.load_data import load
from src.preprocessing.preprocessing import (
    print_null_rows,
    drop_missing_text_rows,
    shuffle_rows,
    remove_stop_words,
    stem_words,
)

In [83]:
spark = SparkSession.builder.appName("NewsReliability").getOrCreate()

In [84]:
data = load(spark)

In [85]:
data.show(10)

+--------------------+--------------------+-------+-----------------+------+
|               title|                text|subject|             date|target|
+--------------------+--------------------+-------+-----------------+------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|     1|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|     1|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|     1|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|     1|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|     1|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|     1|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|     1|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|     1|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|     1|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|     1|

In [86]:
data.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- target: integer (nullable = false)



# Handling Missing Values

In [87]:
print_null_rows(data)

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
|    0|   8|      8|   8|     0|
+-----+----+-------+----+------+



In [88]:
data.filter(data.text.isNull()).show()

+--------------------+----+-------+----+------+
|               title|text|subject|date|target|
+--------------------+----+-------+----+------+
|                   "|NULL|   NULL|NULL|     1|
|Ex-GOP Congressma...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|WATCH: Democratic...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|Trump Gets STOMPE...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|Donald Trump Gets...|NULL|   NULL|NULL|     1|
+--------------------+----+-------+----+------+



In [89]:
data = drop_missing_text_rows(data)

In [90]:
data.filter(data.text.isNull()).show()

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
+-----+----+-------+----+------+



In [91]:
print_null_rows(data)

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
|    0|   0|      0|   0|     0|
+-----+----+-------+----+------+



# Randomize the order of data rows

In [92]:
data = shuffle_rows(data)
data.show(10)

+--------------------+--------------------+------------+-----------------+------+
|               title|                text|     subject|             date|target|
+--------------------+--------------------+------------+-----------------+------+
|China says U.S. v...|BEIJING (Reuters)...|   worldnews|October 11, 2017 |     0|
|HA-HA! RETIRED MA...|It s always a win...|    politics|      Aug 9, 2017|     1|
|China says its ha...|BEIJING (Reuters)...|politicsNews| January 3, 2017 |     0|
| Trump Administra...|Donald Trump s un...|        News|   April 13, 2017|     1|
|U.S. Senate confi...|WASHINGTON (Reute...|politicsNews|  April 27, 2017 |     0|
|Watchdog presses ...|(Reuters) - A gov...|politicsNews|   March 8, 2017 |     0|
| Conservative Whi...|Meryl Streep made...|        News|  January 9, 2017|     1|
|Trump attacks Mit...|BEDMINSTER, N.J. ...|politicsNews| August 10, 2017 |     0|
|Chicago schools w...|CHICAGO (Reuters)...|politicsNews|  April 12, 2016 |     0|
| WATCH: Hypocri

## Feature Creating

In [93]:
# combine title and text column to make news content
data = data.withColumn('content', concat(data['title'], data['text']))
data.show(3)

+--------------------+--------------------+------------+-----------------+------+--------------------+
|               title|                text|     subject|             date|target|             content|
+--------------------+--------------------+------------+-----------------+------+--------------------+
|China says U.S. v...|BEIJING (Reuters)...|   worldnews|October 11, 2017 |     0|China says U.S. v...|
|HA-HA! RETIRED MA...|It s always a win...|    politics|      Aug 9, 2017|     1|HA-HA! RETIRED MA...|
|China says its ha...|BEIJING (Reuters)...|politicsNews| January 3, 2017 |     0|China says its ha...|
+--------------------+--------------------+------------+-----------------+------+--------------------+
only showing top 3 rows



In [94]:
data.select("content").show(n = 5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Drop columns

In [95]:
columns_to_drop = ["date", "title", "subject", "text"]
data = data.drop(*columns_to_drop)
data.show(2)

+------+--------------------+
|target|             content|
+------+--------------------+
|     0|China says U.S. v...|
|     1|HA-HA! RETIRED MA...|
+------+--------------------+
only showing top 2 rows



# Duplicate Rows

In [96]:
duplicate_rows = data.groupBy(data.columns).count().filter("count > 1")

print("Duplicates", duplicate_rows.count())
print("Total", data.count())
print("Distinct", data.distinct().count())

Duplicates 5305
Total 44899


Distinct 39106


In [97]:
duplicate_rows.show(10)

+------+--------------------+-----+
|target|             content|count|
+------+--------------------+-----+
|     1|JUST IN: SUSPECTE...|    2|
|     1|YOU’LL NEVER GUES...|    2|
|     1|HOUSE INTEL Slaps...|    2|
|     1|FLASHBACK: CNN HO...|    2|
|     1|PRICELESS! WATCH ...|    2|
|     1|CNN’S FAKE NEWS B...|    2|
|     1|WATCH: MEGHAN MCC...|    2|
|     1|“MAXINE WATERS IN...|    2|
|     1|HEY CNN…REMEMBER ...|    2|
|     1|HOW PRESIDENT TRU...|    2|
+------+--------------------+-----+
only showing top 10 rows



In [98]:
data = data.dropDuplicates()
data.count()

39106

# Text Transform 

### 1. Lower Case

In [99]:
# convert to all lower case
data = data.withColumn("content", lower(col("content")))
data.show(5)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1| trump bucks gop ...|
|     1| australia prime ...|
|     1| mccain f*cked ov...|
|     1| american psychoa...|
|     1| senate gives tru...|
+------+--------------------+
only showing top 5 rows



### 2. Remove Special Characters 
- like ! @

In [100]:
data = data.withColumn("content", regexp_replace("content", "[^a-zA-Z0-9\\s]", ""))
data.show(5)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1| trump bucks gop ...|
|     1| australia prime ...|
|     1| mccain fcked ove...|
|     1| american psychoa...|
|     1| senate gives tru...|
+------+--------------------+
only showing top 5 rows



### 3. Remove Stopwords

In [101]:
remove_stop_words_udf = udf(remove_stop_words, StringType())
data = data.withColumn("content", remove_stop_words_udf("content"))
data.show(10)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1|two somali muslim...|
|     0|us consumer watch...|
|     0|trump pushes us l...|
|     1|video democrats f...|
|     0|venezuela preside...|
|     1|mainstream media ...|
|     0|may nudges uk tow...|
|     1|trumps vp pick at...|
|     0|us senator hatch ...|
|     1|black politicians...|
+------+--------------------+
only showing top 10 rows



### 4. Stemming words

In [102]:
stem_words_udf = udf(stem_words, StringType())
data = data.withColumn("content", stem_words_udf("content"))
data.show(5)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1|two somali muslim...|
|     0|us consum watchdo...|
|     0|trump push us lab...|
|     1|video democrat fr...|
|     0|venezuela presid ...|
+------+--------------------+
only showing top 5 rows



## Train Test Split

In [103]:
train_data, test_data = data.randomSplit(weights=[0.8, 0.2], seed=100)

In [104]:
# print(train_data.count())
# print(test_data.count())

### Count vectorization
- count how many times each word show up in the text - frequency count
-  transforms the count into DTM(Document Term Matrix)
- Matices are store as sparse matrix to save space, as many values are 0
- Bag of Words

### TF-IDF 
- Term Frequency - Inverse Document Frequency
- address issue with document frequency
- instead of filling DMT with word frequency count, it calculates TD-IDF
- TF: raw count of a term in the document
- but words that are common accross documents? IDF !
- IDF: diminishes the weight of term that occur frequently in the document set and increases the weight of term that occess rarely. 

### TfidfVectorizer
- does both: create bag of word and convert to tfidf

### Tokenization
- split the content into words array

In [105]:
tokenizer = Tokenizer(inputCol="content", outputCol="tokenized")

### HashingTf 
- for term frequencies

In [106]:
hashingTf = HashingTF(inputCol="tokenized", outputCol="tfhashed", numFeatures=100)



### IDF
- for inverse document frequency



In [107]:
idf = IDF(inputCol="tfhashed", outputCol="features")

### Vector Assembler

In [108]:
feature_assembler = VectorAssembler(inputCols=["features"], outputCol="features_vector")

## Pipeline

In [109]:
pipeline = Pipeline(stages=[tokenizer, hashingTf, idf, feature_assembler])

In [110]:
pipeline_model = pipeline.fit(train_data)

In [111]:
train_transformed = pipeline_model.transform(train_data)
test_transformed = pipeline_model.transform(test_data)

In [112]:
test_transformed.show(2)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|target|             content|           tokenized|            tfhashed|            features|     features_vector|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     0|900 state depart ...|[900, state, depa...|(100,[8,10,12,16,...|(100,[8,10,12,16,...|(100,[8,10,12,16,...|
|     0|act sec chair sig...|[act, sec, chair,...|(100,[0,1,2,3,4,5...|(100,[0,1,2,3,4,5...|[0.86299586540780...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [117]:
train_transformed.show(2)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|target|             content|           tokenized|            tfhashed|            features|     features_vector|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     0|17yearold danish ...|[17yearold, danis...|(100,[0,1,8,14,15...|(100,[0,1,8,14,15...|(100,[0,1,8,14,15...|
|     0|2 hous republican...|[2, hous, republi...|(100,[2,4,5,11,13...|(100,[2,4,5,11,13...|(100,[2,4,5,11,13...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



### Save data locally

In [114]:
path = "data/processed"
if not os.path.exists(path):
    os.makedirs(path)

In [115]:
# training set
train_transformed.coalesce(1).toPandas().to_csv("data/processed/train_transformed.csv", index=False)

# testing set
train_transformed.coalesce(1).toPandas().to_csv("data/processed/test_transformed.csv", index=False)